In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

2024-03-02 08:18:49.446461: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_X =  [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X)) # 단어 벡터의 차원 5, 문장의 길이 4
# 4번의 시점(timesteps)가 존재하고, 각 시점마다 5개의 단어 벡터가 입력으로 사용됨.

(4, 5)


In [4]:
# 단 RNN은 2D텐서가 아니고 3D 텐서를 입력으로 받으므로 배치 크기 1을 추가
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape) # batch_size는 1이고 RNN이 학번에 학습하는 양

(1, 4, 5)


In [8]:
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_states, last_state, last_cell_state = lstm(train_X)

print('hidden states = {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state = {}, shape: {}'.format(last_state, last_state.shape))
print('last_cell_state = {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

# simpleRNN과 다르게 3개의 출력을 반환, return_sequences가 False이므로 첫번째는 마지막 시점의 은닉상태
# return_state가 True인경우 마지막 셀의 상태까지 반환한다는점

hidden states = [[-0.57475626  0.25164804 -0.04415815]], shape: (1, 3)
last hidden state = [[-0.57475626  0.25164804 -0.04415815]], shape: (1, 3)
last_cell_state = [[-1.4341611   0.5343475  -0.26574236]], shape: (1, 3)


In [10]:
#return sequences를 True로 바꿔보자
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_state, last_cell_state = lstm(train_X)

print('hidden states = {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state = {}, shape: {}'.format(last_state, last_state.shape))
print('last_cell_state = {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

# 모든 시점의 은닉상태가 출력되는 거 외에는 동일하다.

hidden states = [[[-0.5541065   0.18113732 -0.48967132]
  [-0.4707675  -0.03360611 -0.75242496]
  [-0.26216856 -0.03199646 -0.5714816 ]
  [-0.35336885 -0.01527879 -0.7253603 ]]], shape: (1, 4, 3)
last hidden state = [[-0.35336885 -0.01527879 -0.7253603 ]], shape: (1, 3)
last_cell_state = [[-0.92141896 -0.04457372 -1.7044477 ]], shape: (1, 3)


In [15]:
# 양방향 LSTM의 출력값을 확인해보자.
# return_sequences가 True인 경우와 False인 경우에 대해서 은닉 상태의 값이 어떻게 바뀌는지 직접 비교하기 위해서 출력되는 은닉 상태의 값을 고정

k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

# return_sequences가 False이고, return_state가 True인 경우
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                       kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape)) # 아래 두개를 연결 한 값이 이 값
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape)) # 정방향 LSTM의 마지막 시점의 은닉 상태
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape)) # 역방향 LSTM의 첫번째 시점의 은닉 상태

# 5개의 출력값이 존재
# return_state가 True인 경우에는 정방향 LSTM의 은닉 상태와 셀 상태, 역방향 LSTM의 은닉 상태와 셀 상태 4가지를 반환하기 때문
# 첫번째 출력값이 (1, 6)인 이유는 return_sequences가 False인 경우 정방향 LSTM의 마지막시점의 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉상태가 연결

hidden states : [[0.6303138 0.6303138 0.6303138 0.7038734 0.7038734 0.7038734]], shape: (1, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


In [17]:
# return_sequences=True인 경우
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                       kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape)) # 모든 시점의 은닉 상태
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape)) # 정방향 LSTM의 마지막 시점의 은닉 상태
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape)) # 역방향 LSTM의 첫번째 시점의 은닉 상태
# hidden states를 보면 역방향 첫번째 시점의 은닉 상태는 더이상 정방향 마지막 시점의 은닉 상태와 연결되지 않은 것을 볼 수있다
# 정방향 첫번째 시점과 연결되어 있다. (모든 시점의 은닉 상태를 출력하므로)

hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.55111325 0.55111325 0.55111325 0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.6303138  0.6303138  0.6303138  0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)
